In [1]:

    var SparkConf = require('eclairjs/SparkConf');
    var SparkContext = require('eclairjs/SparkContext');
    var sparkConf = new SparkConf().setAppName("Example");
    var sc = new SparkContext(sparkConf);
    var SQLContext = require('eclairjs/sql/SQLContext');
    var StructField = require("eclairjs/sql/types/StructField");
    var StructType = require("eclairjs/sql/types/StructType");
    var Metadata = require("eclairjs/sql/types/Metadata");
    var RowFactory = require("eclairjs/sql/RowFactory");
    var Vectors = require("eclairjs/mllib/linalg/Vectors");
    var LDA = require("eclairjs/ml/clustering/LDA");
    var Vector = require("eclairjs/mllib/linalg/Vector");
    var VectorUDT = require("eclairjs/mllib/linalg/VectorUDT");


In [2]:
    var sqlContext = new SQLContext(sc);

    var inputFile = "examples/data/mllib/sample_lda_data.txt";


 Loads data


In [3]:
    var points = sc.textFile(inputFile).map(function(line, RowFactory, Vectors){
        var tok = line.split(" ");
        var point = [];
        for (var i = 0; i < tok.length; ++i) {
            point[i] = parseFloat(tok[i]);
        }
        var points = Vectors.dense(point);
        return RowFactory.create(points);
    }, [RowFactory, Vectors]);
    var fields = [new StructField("features", new VectorUDT(), false, Metadata.empty())];
    var schema = new StructType(fields);
    var dataset = sqlContext.createDataFrame(points, schema);


 Trains a LDA model


In [4]:
    var lda = new LDA()
        .setK(10)
        .setMaxIter(10);
    var model = lda.fit(dataset);
    var ret = {};
    ret.logLikelihood = model.logLikelihood(dataset);
    ret.logPerplexity = model.logPerplexity(dataset);


 Shows the result


In [5]:
    ret.topics = model.describeTopics(3);

    ret.transformed = model.transform(dataset);

var result = ret;
    print(result.logLikelihood);
    print(result.logPerplexity);
    result.topics.show(false);
    result.transformed.show(false);

    sc.stop();
